## FASE 2: PRÉ-PROCESSAMENTO DOS DADOS

#### Esta fase justifica as transformações feitas para tornar o modelo de Machine Learning viável.

### 2.1: Pré-processamento e Escalonamento de Dados

## Fase 2: Preparação dos Dados (Data Preparation)

### Justificativa

Após entender a estrutura e o comportamento dos dados na Fase 1 (EDA), esta fase tem como objetivo preparar o dataset para a modelagem de forma **consistente, reproduzível e livre de vazamento de dados**. Em vez de aplicar transformações “soltas” diretamente nos DataFrames, todo o pré-processamento é encapsulado em um objeto único (`preprocessor.joblib`), que será reutilizado na fase de modelagem (notebook 3) e, posteriormente, no deployment (notebook 4).

Essa abordagem segue as boas práticas de projetos de Machine Learning em produção:

- garante que as mesmas transformações sejam aplicadas em treino, validação, teste e produção;
- reduz risco de data leakage (parâmetros de imputação e escala são aprendidos apenas no conjunto de treino);
- facilita integração com pipelines de validação cruzada e tuning de hiperparâmetros.

---

### Etapas

1. **Carregamento do dataset bruto**  
   - Leitura do arquivo `creditcard.csv` diretamente do repositório do Kaggle.  
   - Verificação das dimensões iniciais do dataset.

2. **Definição formal da variável-alvo e das features**  
   - Definição de `Class` como coluna alvo (`target_col`).  
   - Identificação de todas as colunas numéricas.  
   - Remoção explícita da coluna alvo da lista de features numéricas.  
   - Criação de listas `numeric_cols` e `categorical_cols` que serão a referência oficial do projeto.

3. **Construção do pré-processador oficial (`preprocessor`)**  
   - Imputação de valores ausentes em variáveis numéricas com a mediana (`SimpleImputer(strategy="median")`).  
   - Escalonamento robusto (`RobustScaler`) para reduzir o impacto de outliers em features como `Time` e `Amount`.  
   - (Opcional) Imputação e codificação one-hot para variáveis categóricas, caso existam em versões futuras do dataset.  
   - Encapsulamento dessas transformações em um `ColumnTransformer`, compatível com `Pipeline`, `Cross-Validation` e `RandomizedSearchCV`.

4. **Persistência do pré-processador**  
   - Salvamento do objeto `preprocessor` em `artifacts/preprocessor.joblib`.  
   - Salvamento de metadados de pré-processamento (nome da coluna alvo, listas de colunas numéricas e categóricas) em `artifacts/preprocessing_metadata.json`.  
   - Esses artefatos serão carregados na Fase 3 (Modelagem) para construir os pipelines completos de treino e tuning, garantindo consistência entre notebooks e ambientes.

---

### Benefícios

- **Consistência entre as fases do projeto**: o mesmo pré-processamento é utilizado em todas as etapas (treino, validação, teste, produção), evitando discrepâncias entre notebooks e scripts.
- **Redução de vazamento de dados (data leakage)**: parâmetros de imputação e escala são aprendidos apenas nos dados de treino, quando o `preprocessor` é usado em conjunto com validação cruzada e pipelines completos.
- **Modularidade e reuso**: o objeto `preprocessor` pode ser acoplado a diferentes modelos (Logistic Regression, Random Forest, XGBoost, LightGBM, CatBoost) sem duplicação de código.
- **Facilidade de experimentação**: qualquer alteração nas regras de pré-processamento (imputação, escala, encoding) é feita em um único ponto (`pipeline_new.py`), refletindo automaticamente em todas as fases do projeto.
- **Preparação para produção**: o uso de artefatos versionáveis (`joblib` + `metadata.json`) aproxima o notebook de um fluxo real de MLOps, facilitando o deploy e o monitoramento do modelo final.


### Célula 1 – Imports e ligação com pipeline_new.py

In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys
import joblib # Para salvar os objetos transformados

project_root = r"C:\Users\debor\OneDrive\Github\FraudSense"

if project_root not in sys.path:
    sys.path.append(project_root)

from pipeline_new import build_preprocessor, save_pipeline

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler
...


Ellipsis

### Célula 2 – Carregamento dos dados com load_data()

In [8]:
# Célula 2 Carregamento dos Dados
def load_data():
    """Carrega o dataset original"""
    # Se você já baixou no notebook 1, o arquivo deve estar em cache ou na pasta local
    # Ajuste o caminho se necessário, ou use o kagglehub novamente
    import kagglehub
    path = kagglehub.dataset_download("mlg-ulb/creditcardfraud")
    file_path = os.path.join(path, 'creditcard.csv')
    return pd.read_csv(file_path)

df = load_data()
print(f"Dados carregados: {df.shape}")

Dados carregados: (284807, 31)


### Célula 3 – Definição de colunas oficiais

In [9]:
# Célula 3: definição de colunas para o pipeline oficial

# Target
target_col = "Class"

# Features numéricas (no dataset creditcard.csv, praticamente todas são numéricas)
numeric_cols = df.select_dtypes(include=["number"]).columns.tolist()

# Remover a coluna alvo da lista de numéricas
if target_col in numeric_cols:
    numeric_cols.remove(target_col)

# Se você tiver colunas categóricas em algum momento futuro, coloque aqui
categorical_cols = []  # no creditcard.csv original não há categorias

print("Colunas numéricas usadas no pipeline:")
print(numeric_cols[:10], "... (total:", len(numeric_cols), ")")
print("Colunas categóricas usadas no pipeline:")
print(categorical_cols)


Colunas numéricas usadas no pipeline:
['Time', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9'] ... (total: 30 )
Colunas categóricas usadas no pipeline:
[]


### Célula 4 – Construção do pré-processador oficial

In [10]:
# Célula 4: construção do pré-processador oficial para o projeto

from pipeline_new import build_preprocessor

preprocessor = build_preprocessor(
    numeric_cols=numeric_cols,
    categorical_cols=categorical_cols,
    numeric_strategy="median",  # igual ao que você já vinha usando
    scale="robust"              # coerente com seu uso de RobustScaler
)

print("Pré-processador criado com sucesso!")
print(type(preprocessor))
print("Transformers definidos:")
for name, trans, cols in preprocessor.transformers:
    print(f" • {name}: {type(trans).__name__} -> {len(cols)} colunas")


Pré-processador criado com sucesso!
<class 'sklearn.compose._column_transformer.ColumnTransformer'>
Transformers definidos:
 • num: Pipeline -> 30 colunas


### Célula 5 – Salvando o pré-processador oficial

In [11]:
# Célula 5: salvar o pré-processador oficial

from pipeline_new import save_pipeline

os.makedirs("artifacts", exist_ok=True)

preprocessor_path = os.path.join("artifacts", "preprocessor.joblib")
save_pipeline(preprocessor, preprocessor_path)

print("\nPré-processador salvo com sucesso!")
print("Caminho:", preprocessor_path)
print("Este é o objeto que será usado na fase de modelagem (notebook 3).")


Pipeline salvo em artifacts\preprocessor.joblib

Pré-processador salvo com sucesso!
Caminho: artifacts\preprocessor.joblib
Este é o objeto que será usado na fase de modelagem (notebook 3).


## ⚠️IMPORTANTE:

### As funções abaixo (stratified_split_data, robust_scaling, balance_classes, etc.)  foram utilizadas no desenvolvimento e entendimento do pré-processamento,  mas o pipeline final de produção está centralizado no módulo `pipeline_new.py`  e no objeto `preprocessor` salvo em artifacts/preprocessor.joblib.


### Análise Inicial para Definir Estratégia de Pré-processamento

In [ ]:
# Célula 8: Análise Inicial para Definir Estratégia de Pré-processamento (SEM DATA LEAKAGE)
def pre_processing_analysis(df):
    """Analisa os dados para definir a melhor estratégia de pré-processamento"""
    
    print("=" * 60)
    print(" ANÁLISE PARA DEFINIÇÃO DE PRÉ-PROCESSAMENTO")
    print("=" * 60)
    
    # Análise das features numéricas
    print("\n **ANÁLISE DAS FEATURES NUMÉRICAS:**")
    
    # Separar features PCA das originais
    pca_features = [f'V{i}' for i in range(1, 29)]
    original_features = ['Time', 'Amount']
    
    print(f"• Features PCA (V1-V28): {len(pca_features)}")
    print(f"• Features originais: {original_features}")
    
    # Estatísticas das features originais
    print(f"\n **ESTATÍSTICAS DAS FEATURES ORIGINAIS:**")
    stats_originais = df[original_features].describe()
    print(stats_originais)
    
    # Verificar distribuição das features PCA
    print(f"\n **DISTRIBUIÇÃO DAS FEATURES PCA:**")
    pca_stats = df[pca_features].describe().T[['mean', 'std', 'min', 'max']]
    print(pca_stats.head(8))  # Mostrar apenas as primeiras
    
    # **REMOVIDA a análise de correlação com target aqui**
    # Isso será feito APENAS no conjunto de treino depois da divisão
    
    print(f"\n **IMPORTANTE:** Análise de correlação será realizada")
    print("  apenas no conjunto de TREINO para evitar data leakage")
    
    return pca_features, original_features

# Executar análise
pca_features, original_features = pre_processing_analysis(df)

### Divisão Estratificada dos Dados

In [ ]:
# Célula 9: Divisão Estratificada dos Dados
def stratified_split_data(df, test_size=0.2, random_state=42):
    """
    Divide os dados de forma estratificada mantendo a proporção de classes
    """
    
    print("=" * 50)
    print(" DIVISÃO ESTRATIFICADA DOS DADOS")
    print("=" * 50)
    
    # Separar features e target
    X = df.drop('Class', axis=1)
    y = df['Class']
    
    print(f"• Shape original: {X.shape}")
    print(f"• Proporção da classe 1 (fraude): {y.mean():.4%}")
    
    # Divisão estratificada
    X_temp, X_test, y_temp, y_test = train_test_split(
        X, y, 
        test_size=test_size, 
        stratify=y, 
        random_state=random_state
    )
    
    # Divisão do temporário em treino e validação
    X_train, X_val, y_train, y_val = train_test_split(
        X_temp, y_temp, 
        test_size=0.25,  # 0.25 * 0.8 = 0.2 do total
        stratify=y_temp, 
        random_state=random_state
    )
    
    print(f"\n **DIVISÃO CONCLUÍDA:**")
    print(f"• Conjunto de Treino: {X_train.shape} ({len(X_train)/len(X):.1%})")
    print(f"• Conjunto de Validação: {X_val.shape} ({len(X_val)/len(X):.1%})")
    print(f"• Conjunto de Teste: {X_test.shape} ({len(X_test)/len(X):.1%})")
    
    print(f"\n **DISTRIBUIÇÃO DAS CLASSES NOS CONJUNTOS:**")
    for nome, conjunto in [('Treino', y_train), ('Validação', y_val), ('Teste', y_test)]:
        prop_fraude = conjunto.mean()
        print(f"• {nome}: {conjunto.shape[0]:,} amostras ({prop_fraude:.4%} fraudes)")
    
    return X_train, X_val, X_test, y_train, y_val, y_test

# Executar divisão
X_train, X_val, X_test, y_train, y_val, y_test = stratified_split_data(df)

### Escalonamento Robustos das Features

In [ ]:
# Célula 10: Escalonamento Robustos das Features
def robust_scaling(X_train, X_val, X_test, original_features):
    """
    Aplica escalonamento robusto às features originais 'Time' e 'Amount'
    """
    
    print("=" * 50)
    print(" ESCALONAMENTO ROBUSTO DAS FEATURES")
    print("=" * 50)
    
    # Criar cópias para não modificar os originais
    X_train_scaled = X_train.copy()
    X_val_scaled = X_val.copy()
    X_test_scaled = X_test.copy()
    
    # Inicializar scalers robustos (menos sensível a outliers)
    scaler_time = RobustScaler()
    scaler_amount = RobustScaler()
    
    print(" **APLICANDO ESCALONAMENTO:**")
    
    # Escalonar feature 'Time'
    X_train_scaled['Time'] = scaler_time.fit_transform(X_train[['Time']])
    X_val_scaled['Time'] = scaler_time.transform(X_val[['Time']])
    X_test_scaled['Time'] = scaler_time.transform(X_test[['Time']])
    print("• Feature 'Time' escalonada com RobustScaler")
    
    # Escalonar feature 'Amount'
    X_train_scaled['Amount'] = scaler_amount.fit_transform(X_train[['Amount']])
    X_val_scaled['Amount'] = scaler_amount.transform(X_val[['Amount']])
    X_test_scaled['Amount'] = scaler_amount.transform(X_test[['Amount']])
    print("• Feature 'Amount' escalonada com RobustScaler")
    
    # Verificar estatísticas após escalonamento
    print(f"\n **ESTATÍSTICAS APÓS ESCALONAMENTO (TREINO):**")
    stats_apos = X_train_scaled[original_features].describe()
    print(stats_apos)
    
    print(f"\n **JUSTIFICATIVA TÉCNICA:**")
    print("• RobustScaler: Remove mediana e escala com IQR, robusto a outliers")
    print("• Preserva distribuição original melhor que StandardScaler")
    print("• Ideal para dados financeiros com outliers")
    
    return X_train_scaled, X_val_scaled, X_test_scaled, scaler_time, scaler_amount

# Aplicar escalonamento
X_train_scaled, X_val_scaled, X_test_scaled, scaler_time, scaler_amount = robust_scaling(
    X_train, X_val, X_test, original_features
)

### Análise de Correlação APENAS no Treino (SEM DATA LEAKAGE)

In [ ]:
# Célula 10.5: Análise de Correlação APENAS no Treino (SEM DATA LEAKAGE)
def analyze_training_correlations(X_train_scaled, y_train):
    """Analisa correlações APENAS no conjunto de treino"""
    
    print("=" * 60)
    print(" ANÁLISE DE CORRELAÇÃO NO CONJUNTO DE TREINO")
    print("=" * 60)
    print(" **EVITANDO DATA LEAKAGE**")
    
    # Combinar features e target do treino
    train_data = X_train_scaled.copy()
    train_data['Class'] = y_train.values
    
    # Calcular correlações
    correlations = train_data.corr()['Class'].sort_values(ascending=False)
    
    # Top 10 features mais correlacionadas
    print("\n **TOP 10 FEATURES MAIS CORRELACIONADAS COM FRAUDE:**")
    print("(Ordem decrescente de importância)")
    
    top_positive = correlations[1:11]  # Excluir a correlação perfeita com ela mesma
    top_negative = correlations[-10:]
    
    print("\nCorrelações POSITIVAS (mais associadas a fraude):")
    for feature, corr in top_positive.head(5).items():
        print(f"  • {feature}: {corr:.4f}")
    
    print("\nCorrelações NEGATIVAS (menos associadas a fraude):")
    for feature, corr in top_negative.tail(5).items():
        print(f"  • {feature}: {corr:.4f}")
    
    # Visualização
    plt.figure(figsize=(12, 8))
    top_corr = pd.concat([top_positive.head(5), top_negative.tail(5)])
    colors = ['coral' if x > 0 else 'skyblue' for x in top_corr]
    
    ax = top_corr.sort_values().plot(kind='barh', color=colors)
    plt.title('Top 10 Features Mais Correlacionadas com Fraude\n(Conjunto de Treino)', 
              fontsize=14, fontweight='bold')
    plt.xlabel('Coeficiente de Correlação')
    plt.axvline(x=0, color='black', linestyle='--', alpha=0.5)
    plt.tight_layout()
    plt.show()
    
    # Insights
    print(f"\n **INSIGHTS:**")
    print(f"• Feature mais associada à fraude: {correlations.index[1]} ({correlations[1]:.4f})")
    print(f"• Feature menos associada à fraude: {correlations.index[-1]} ({correlations[-1]:.4f})")
    print(f"• Total de features com correlação > 0.1: {(correlations.abs() > 0.1).sum()}")
    
    return correlations

# Executar análise APÓS o escalonamento
correlations = analyze_training_correlations(X_train_scaled, y_train)

### Técnicas de Balanceamento de Classes

In [ ]:
# Célula 11: Técnicas de Balanceamento de Classes
def balance_classes(X_train_scaled, y_train, strategy='smote'):
    """
    Aplica técnicas de balanceamento para lidar com o desbalanceamento
    """
    
    print("=" * 60)
    print(" TÉCNICAS DE BALANCEAMENTO DE CLASSES")
    print("=" * 60)
    
    print(f" **DISTRIBUIÇÃO ANTES DO BALANCEAMENTO:**")
    unique, counts = np.unique(y_train, return_counts=True)
    for classe, count in zip(unique, counts):
        tipo = "FRAUDE" if classe == 1 else "NORMAL"
        print(f"• Classe {classe} ({tipo}): {count:,} amostras")
    
    # Diferentes estratégias de balanceamento
    if strategy == 'smote':
        print(f"\n **APLICANDO SMOTE (Synthetic Minority Over-sampling Technique):**")
        smote = SMOTE(
            random_state=42,
            sampling_strategy='auto',  # Balanceia para 50%-50%
            k_neighbors=5
        )
        X_balanced, y_balanced = smote.fit_resample(X_train_scaled, y_train)
        technique = "SMOTE"
        
    elif strategy == 'undersample':
        print(f"\n **APLICANDO UNDERSAMPLING (Subamostragem da Maioria):**")
        undersampler = RandomUnderSampler(
            random_state=42,
            sampling_strategy='auto'
        )
        X_balanced, y_balanced = undersampler.fit_resample(X_train_scaled, y_train)
        technique = "Undersampling"
        
    elif strategy == 'combine':
        print(f"\n **APLICANDO COMBINAÇÃO (SMOTE + Undersampling):**")
        # Primeiro oversampling com SMOTE
        smote = SMOTE(sampling_strategy=0.1, random_state=42)
        X_temp, y_temp = smote.fit_resample(X_train_scaled, y_train)
        
        # Depois undersampling
        undersampler = RandomUnderSampler(sampling_strategy='auto', random_state=42)
        X_balanced, y_balanced = undersampler.fit_resample(X_temp, y_temp)
        technique = "SMOTE + Undersampling"
    
    else:
        # Sem balanceamento
        print(f"\n  **SEM BALANCEAMENTO (apenas para comparação):**")
        X_balanced, y_balanced = X_train_scaled.copy(), y_train.copy()
        technique = "Sem balanceamento"
    
    print(f"\n **DISTRIBUIÇÃO APÓS {technique.upper()}:**")
    unique_after, counts_after = np.unique(y_balanced, return_counts=True)
    for classe, count in zip(unique_after, counts_after):
        tipo = "FRAUDE" if classe == 1 else "NORMAL"
        print(f"• Classe {classe} ({tipo}): {count:,} amostras")
    
    print(f"\n **ESTATÍSTICAS DO BALANCEAMENTO:**")
    print(f"• Amostras antes: {len(X_train_scaled):,}")
    print(f"• Amostras depois: {len(X_balanced):,}")
    print(f"• Proporção fraudes antes: {y_train.mean():.4%}")
    print(f"• Proporção fraudes depois: {y_balanced.mean():.4%}")
    
    return X_balanced, y_balanced, technique

# Aplicar SMOTE (estratégia recomendada)
X_balanced, y_balanced, technique_used = balance_classes(X_train_scaled, y_train, strategy='smote')

### Seleção de Features Baseada em Importância

In [ ]:
# Célula 12: Seleção de Features Baseada em Importância
def feature_selection(X_balanced, y_balanced, X_val_scaled, X_test_scaled, k_features=20):
    """
    Seleciona as k features mais importantes usando teste ANOVA F
    """
    
    print("=" * 50)
    print(" SELEÇÃO DE FEATURES")
    print("=" * 50)
    
    print(f" **FEATURES DISPONÍVEIS:** {X_balanced.shape[1]}")
    
    # Seleção de features usando ANOVA F-value
    selector = SelectKBest(score_func=f_classif, k=k_features)
    
    print(f"\n **SELECIONANDO AS {k_features} MELHORES FEATURES...**")
    X_selected = selector.fit_transform(X_balanced, y_balanced)
    X_val_selected = selector.transform(X_val_scaled)
    X_test_selected = selector.transform(X_test_scaled)
    
    # Obter scores e nomes das features selecionadas
    feature_scores = selector.scores_
    feature_names = X_balanced.columns
    selected_mask = selector.get_support()
    
    # Criar DataFrame com scores
    feature_importance = pd.DataFrame({
        'Feature': feature_names,
        'F_Score': feature_scores,
        'Selecionada': selected_mask
    }).sort_values('F_Score', ascending=False)
    
    print(f"\n **TOP {k_features} FEATURES SELECIONADAS:**")
    print(feature_importance[feature_importance['Selecionada'] == True][['Feature', 'F_Score']].head(k_features))
    
    print(f"\n **RESUMO DA SELEÇÃO:**")
    print(f"• Features originais: {X_balanced.shape[1]}")
    print(f"• Features selecionadas: {X_selected.shape[1]}")
    print(f"• Features removidas: {X_balanced.shape[1] - X_selected.shape[1]}")
    
    # Plotar importância das features
    plt.figure(figsize=(12, 8))
    features_selecionadas = feature_importance[feature_importance['Selecionada'] == True].head(15)
    features_nao_selecionadas = feature_importance[feature_importance['Selecionada'] == False].head(10)
    
    plt.subplot(1, 2, 1)
    plt.barh(features_selecionadas['Feature'], features_selecionadas['F_Score'], color='skyblue')
    plt.title('Top 15 Features Selecionadas\n(maior F-Score)', fontweight='bold')
    plt.xlabel('F-Score (ANOVA)')
    plt.gca().invert_yaxis()
    
    plt.subplot(1, 2, 2)
    plt.barh(features_nao_selecionadas['Feature'], features_nao_selecionadas['F_Score'], color='lightcoral')
    plt.title('Top 10 Features Não Selecionadas\n(menor F-Score)', fontweight='bold')
    plt.xlabel('F-Score (ANOVA)')
    plt.gca().invert_yaxis()
    
    plt.tight_layout()
    plt.show()
    
    return X_selected, X_val_selected, X_test_selected, selector, feature_importance

# Aplicar seleção de features
X_selected, X_val_selected, X_test_selected, selector, feature_importance = feature_selection(
    X_balanced, y_balanced, X_val_scaled, X_test_scaled, k_features=20
)

### Pipeline Completo de Pré-processamento

In [ ]:
# Célula 13: Pipeline Completo de Pré-processamento
def create_preprocessing_pipeline():
    """
    Cria um pipeline reprodutível para pré-processamento
    """
    
    print("=" * 50)
    print(" PIPELINE DE PRÉ-PROCESSAMENTO")
    print("=" * 50)
    
    # Definir steps do pipeline
    preprocessing_steps = {
        '1. Divisão Estratificada': 'train_test_split com stratify',
        '2. Escalonamento RobustScaler': 'Time e Amount',
        '3. Balanceamento SMOTE': 'Synthetic Minority Over-sampling',
        '4. Seleção de Features': 'SelectKBest com ANOVA F-test',
        '5. Preservação Conjuntos': 'Validação e Teste sem vazamento'
    }
    
    print(" **STEPS DO PIPELINE:**")
    for step, descricao in preprocessing_steps.items():
        print(f"  • {step}: {descricao}")
    
    # Criar dicionário com objetos para reproduzibilidade
    preprocessing_objects = {
        'scalers': {
            'time_scaler': scaler_time,
            'amount_scaler': scaler_amount
        },
        'selector': selector,
        'feature_importance': feature_importance,
        'technique_used': technique_used
    }
    
    print(f"\n **OBJETOS PARA REPRODUTIBILIDADE:**")
    for obj_name, obj in preprocessing_objects.items():
        if obj_name != 'feature_importance':  # Não printar o DataFrame completo
            print(f"  • {obj_name}: {type(obj).__name__}")
    
    return preprocessing_objects

# Criar pipeline
preprocessing_objects = create_preprocessing_pipeline()

### Resumo Final do Pré-processamento

In [ ]:
# Célula 14: Resumo Final do Pré-processamento
def preprocessing_summary(X_selected, X_val_selected, X_test_selected, y_balanced, y_val, y_test):
    """
    Fornece um resumo completo do pré-processamento
    """
    
    print("=" * 60)
    print(" RESUMO FINAL - PRÉ-PROCESSAMENTO")
    print("=" * 60)
    
    print("\n **CONFIGURAÇÃO FINAL DOS CONJUNTOS:**")
    
    conjuntos = {
        'Treino (Balanceado)': (X_selected, y_balanced),
        'Validação': (X_val_selected, y_val),
        'Teste': (X_test_selected, y_test)
    }
    
    for nome, (X, y) in conjuntos.items():
        print(f"\n **{nome}:**")
        print(f"  • Shape: {X.shape}")
        print(f"  • Amostras: {X.shape[0]:,}")
        print(f"  • Features: {X.shape[1]}")
        print(f"  • Proporção Fraudes: {y.mean():.4%}")
        print(f"  • Número de Fraudes: {y.sum():,}")
    
    print(f"\n **PRINCIPAIS TRANSFORMAÇÕES APLICADAS:**")
    transformacoes = [
        "Divisão estratificada (80-20)",
        "Escalonamento RobustScaler em 'Time' e 'Amount'", 
        "Balanceamento com SMOTE",
        "Seleção das 20 melhores features (ANOVA F-test)",
        "Preservação de validação/teste sem vazamento"
    ]
    
    for i, transf in enumerate(transformacoes, 1):
        print(f"  {i}. {transf}")
    
    print(f"\n **ESTATÍSTICAS GLOBAIS:**")
    total_amostras = len(X_selected) + len(X_val_selected) + len(X_test_selected)
    print(f"• Total de amostras processadas: {total_amostras:,}")
    print(f"• Features reduzidas de {df.shape[1]-1} para {X_selected.shape[1]}")
    print(f"• Taxa de redução dimensional: {((df.shape[1]-1 - X_selected.shape[1])/(df.shape[1]-1)*100):.1f}%")
    
    print(f"\n **PRÓXIMOS PASSOS - MODELAGEM:**")
    proximos_passos = [
        "1. Treinar modelos na versão balanceada (X_selected, y_balanced)",
        "2. Validar performance na versão original (X_val_selected, y_val)", 
        "3. Testar final no conjunto blind (X_test_selected, y_test)",
        "4. Focar em Recall para detecção de fraudes",
        "5. Usar Stratified K-Fold para validação robusta"
    ]
    
    for passo in proximos_passos:
        print(f"  {passo}")

# Executar resumo
preprocessing_summary(X_selected, X_val_selected, X_test_selected, y_balanced, y_val, y_test)

### Salvamento dos Dados Processados

In [ ]:
# Célula 15: Salvamento dos Dados Processados
def save_processed_data(X_selected, X_val_selected, X_test_selected, y_balanced, y_val, y_test):
    """
    Salva os dados processados para uso na fase de modelagem
    """
    
    print("=" * 50)
    print(" SALVANDO DADOS PROCESSADOS")
    print("=" * 50)
    
    # Criar dicionário com todos os conjuntos
    processed_data = {
        'X_train': X_selected,
        'X_val': X_val_selected, 
        'X_test': X_test_selected,
        'y_train': y_balanced,
        'y_val': y_val,
        'y_test': y_test,
        'feature_names': X_selected.columns if hasattr(X_selected, 'columns') else [f'V{i}' for i in range(X_selected.shape[1])]
    }
    
    # Salvar usando pickle
    import pickle
    
    with open('processed_fraud_data.pkl', 'wb') as f:
        pickle.dump(processed_data, f)
    
    print(" **DADOS SALVOS COM SUCESSO:**")
    print("• Arquivo: 'processed_fraud_data.pkl'")
    print("• Contém: X_train, X_val, X_test, y_train, y_val, y_test, feature_names")
    print("• Pronto para a fase de modelagem!")
    
    return processed_data

# Salvar dados
processed_data = save_processed_data(X_selected, X_val_selected, X_test_selected, y_balanced, y_val, y_test)